# 具体应用：用Paddlepaddle做一个服装广告文案生成器

## 1. 安装相关库

In [1]:
# 检查paddlepaddle是否可以使用GPU
import paddle
paddle.utils.run_check()

Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 GPU.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.


按照数据集要求对其进行整理，格式为“序号\t输入文本\t标签”

In [2]:
import pandas as pd

df = pd.read_excel("data.xlsx")
Keys = df["关键词"].values
Txts = df["文案"].values

with open("format_data.txt", "w",encoding='utf-8') as f:
    for i, k in enumerate(Keys):
        t = Txts[i]
        f.write("{}\t{}\t{}\n".format(i, k, t))

## 2. 调用Paddlehub模型进行预训练

### 模型我们选用ERNIE-GEN模型

论文地址：https://arxiv.org/abs/2001.11314

![](https://ai-studio-static-online.cdn.bcebos.com/9698f8fef8004f579d01ca0d4a8a8255d58a8cc895c44d59b97141c82a23b833)


In [3]:
import paddlehub as hub

module = hub.Module(name="ernie_gen")

result = module.finetune(
    train_path='format_data.txt',
    save_dir="nlg_param",
    max_steps=2000,
    noise_prob=0.1,
    save_interval=400,
    max_encode_len=60,
    max_decode_len=60
)

# 将训练参数打包为hub model
module.export(params_path=result['last_save_path'], module_name="nlg", author="etta")
!hub install nlg

E:\anaconda3\envs\paddle_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E:\anaconda3\envs\paddle_env\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[2023-12-30 06:54:48,915] [    INFO] - Already cached C:\Users\11409\.paddlenlp\models\ernie-1.0\vocab.txt
[2023-12-30 06:54:48,935] [    INFO] - tokenizer config file saved in C:\Users\11409\.paddlenlp\models\ernie-1.0\tokenizer_config.json
[2023-12-30 06:54:48,936] [    INFO] - Special tokens file saved in C:\Users\11409\.paddlenlp\models\ernie-1.0\special_tokens_map.json
[2023-12-30 06:54:48,962] [    INFO] - Already cached C:\Users\11409\.paddlenlp\models\ernie-1.0\ernie_v1_chn_base.pdparams
[2023-12-30 06:54:48,968] [   DEBUG] - init ErnieModel with config

## 3. 运行预测

In [4]:
import paddlehub as hub

module = hub.Module(directory="nlg")

[2023-12-30 07:06:36,311] [    INFO] - Already cached C:\Users\11409\.paddlenlp\models\ernie-1.0\ernie_v1_chn_base.pdparams
[2023-12-30 07:06:36,311] [   DEBUG] - init ErnieModel with config: {'attention_probs_dropout_prob': 0.1, 'hidden_act': 'relu', 'hidden_dropout_prob': 0.1, 'hidden_size': 768, 'initializer_range': 0.02, 'max_position_embeddings': 513, 'num_attention_heads': 12, 'num_hidden_layers': 12, 'type_vocab_size': 2, 'vocab_size': 18000, 'pad_token_id': 0}
[2023-12-30 07:06:36,638] [    INFO] - loading pretrained model from C:\Users\11409\.paddlenlp\models\ernie-1.0\ernie_v1_chn_base.pdparams
[2023-12-30 07:06:37,147] [    INFO] - param:mlm_bias not set in pretrained model, skip
[2023-12-30 07:06:37,148] [    INFO] - param:mlm.weight not set in pretrained model, skip
[2023-12-30 07:06:37,148] [    INFO] - param:mlm.bias not set in pretrained model, skip
[2023-12-30 07:06:37,148] [    INFO] - param:mlm_ln.weight not set in pretrained model, skip
[2023-12-30 07:06:37,148] [  

In [5]:
test_texts = ["显瘦"]
results = module.generate(texts=test_texts, use_gpu=False, beam_width=10)
for result in results[0]:
    print(result)

这款连衣裙采用优质面料，手感细腻顺滑，手感顺滑，垂感好，垂感好，垂感软糯，垂感好，遮肉显瘦，修身显瘦
这款连衣裙采用优质面料，手感细腻顺滑，手感顺滑，垂感好，垂感好，垂感好，遮肉显瘦，修身的版型，遮肉显
这款连衣裙采用优质面料，手感细腻顺滑，手感顺滑，垂感好，垂感好，垂感好，遮肉显瘦，修身显瘦，修身显瘦
这款连衣裙采用优质面料，手感细腻顺滑，手感顺滑，垂感好，垂感好，垂感软糯，垂感好，遮肉显瘦，修身版型
这款连衣裙采用优质面料，手感细腻顺滑，手感顺滑，垂感好，垂感好，垂感软糯，垂感好，遮肉显瘦，修饰身材
这款连衣裙采用优质面料，手感细腻顺滑，手感顺滑，垂感好，垂感好，垂感好，遮肉显瘦，修身显瘦，修身显腿
这款连衣裙采用优质面料，质地轻盈，手感顺滑，垂感好，垂感好，垂感软糯，垂感好，遮肉显瘦，修身显瘦，修
这款连衣裙采用优质面料，手感细腻顺滑，手感顺滑，垂感好，垂感好，垂感软糯，垂感好，遮肉显瘦，修饰身形
这款连衣裙采用优质面料，手感细腻顺滑，手感顺滑，垂感好，垂感好，垂感好，遮肉显瘦，修身的版型设计，上
这款连衣裙采用优质面料，手感细腻顺滑，手感顺滑，垂感好，垂感好，垂感好，遮肉显瘦，修身的版型设计，修


In [6]:
test_texts = ["性感"]
results = module.generate(texts=test_texts, use_gpu=False, beam_width=5)
for result in results[0]:
    print(result)

这款连衣裙采用优质面料，手感细腻顺滑，手感顺滑，垂感好，垂感好，垂感软糯，垂感好，遮肉显瘦，修身显瘦
这款连衣裙采用优质面料，手感细腻顺滑，手感顺滑，垂感好，手感顺滑，垂感好，垂感好，遮肉显瘦，修身显瘦
这款连衣裙采用优质面料，手感细腻顺滑，手感顺滑，垂感好，垂感好，垂感软糯，垂感好，遮肉显瘦，修饰身材
这款连衣裙采用优质面料，手感细腻顺滑，手感顺滑，垂感好，垂感好，垂感软糯，垂感好，遮肉显瘦，修身版型
这款连衣裙采用优质面料，手感细腻顺滑，手感顺滑，垂感好，手感顺滑，垂感好，垂感好，遮肉显瘦，修饰身材


In [7]:
test_texts = ["复古"]
results = module.generate(texts=test_texts, use_gpu=False, beam_width=5)
for result in results[0]:
    print(result)

这款连衣裙采用优质面料，手感细腻顺滑，手感顺滑，垂感好，垂感好，垂感软糯，垂感好，遮肉显瘦，修身显瘦
这款连衣裙采用优质面料，手感细腻顺滑，手感顺滑，垂感好，垂感好，垂感软糯，垂感好，遮肉显瘦，修饰身材
这款连衣裙采用优质面料，手感细腻顺滑，手感顺滑，垂感好，垂感好，垂感软糯，垂感好，遮肉显瘦，修身版型
这款连衣裙采用优质面料，手感细腻顺滑，手感顺滑，垂感好，垂感好，垂感软糯，垂感好，遮肉显瘦，修身剪裁
这款连衣裙采用优质面料，手感细腻顺滑，手感顺滑，垂感好，垂感好，垂感软糯，垂感好，遮肉显瘦，修饰腿部


## 4. 改进方向：

可以看到数据集太少，出现了明显的过拟合现象，可以多搜集一些。